In [45]:
from tqdm import tqdm
import pandas as pd
import requests

In [46]:
url = './make_file/(동)_열선_취약시설_거리.csv'
file_encoding = 'EUC-KR'

In [47]:
data = pd.read_csv(url, encoding=file_encoding)

In [48]:
data['행정동'].value_counts()

행정동
쌍문동    28
신당동    20
정릉동    19
논현동    19
상도동    18
       ..
성산동     1
삼전동     1
양재동     1
대방동     1
천호동     1
Name: count, Length: 148, dtype: int64

In [49]:
data.columns

Index(['중점_위도', '중점_경도', '경사각', '최근접_시설의_평균거리', '시점_위도', '시점_경도', '종점_위도',
       '종점_경도', '길이', '열선여부', '행정동', '주소', '도로명_여부'],
      dtype='object')

In [50]:
for index, row in data.iterrows(): 
    if (row['중점_위도'] == 37.4937594751172) and (row['중점_경도'] == 126.948653212882) and pd.isnull(row['행정동']):
        data.at[index, '행정동'] = '상도동'


In [51]:
data[data['행정동'].isnull()]

,중점_위도,중점_경도,경사각,최근접_시설의_평균거리,시점_위도,시점_경도,종점_위도,종점_경도,길이,열선여부,행정동,주소,도로명_여부


### 카카오

In [52]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
print(open_api_key_file['사이트'].value_counts())

KAKAO_API = open_api_key_file[open_api_key_file['사이트'] == 'kakao_api_key_REST_sub'].values[0][1]  

사이트
공공데이터                     1
kakao_api_key_REST        1
kakao_api_key_REST_sub    1
카카오 JS                    1
국토부v-world                1
GOOGLE_MAP                1
행정안전부                     1
Name: count, dtype: int64


In [53]:
def get_gu_from_dong(input_dong_name) :
    dong_name = "서울시 " +  str(input_dong_name)
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API}"}
    params = {"query": dong_name}
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"API 요청 실패: {response.status_code}, {response.text}")
    
    result = response.json()
    if not result["documents"]:
        raise ValueError("검색 결과 없음")
    
    first_result = result["documents"][0]
    x, y = first_result["x"], first_result["y"]
    
    # 좌표 -> 행정구역 변환
    region_url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
    params = {"x": x, "y": y}
    response = requests.get(region_url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"API 요청 실패: {response.status_code}, {response.text}")
    
    region_result = response.json()
    for region in region_result["documents"]:
        if region["region_type"] == "B":  # 법정동(B) 대신 행정동(H) 확인 가능
            return region["region_2depth_name"]  # 구 정보 반환
    
    raise ValueError("구 정보를 찾을 수 없음")




In [54]:
temp_dong_list = list(data['행정동'].unique())

In [55]:
dong_gu_dictionary = {}

for i in tqdm(range(len(temp_dong_list))) :
    dong_name = temp_dong_list[i]
    gu_name = get_gu_from_dong(dong_name)
    dong_gu_dictionary[dong_name] = gu_name

100%|█████████████████████████████████████████| 148/148 [00:25<00:00,  5.74it/s]


In [56]:
data['행정구역'] = data['행정동']

In [57]:
data['행정구역'] = data['행정구역'].map(dong_gu_dictionary)

In [58]:
data['행정구역'].value_counts()

행정구역
강남구     59
성북구     53
성동구     48
동작구     45
도봉구     36
광진구     33
중구      31
서대문구    26
중랑구     26
관악구     26
마포구     21
용산구     20
서초구     19
양천구     17
금천구     15
종로구     14
강북구     14
강서구     13
동대문구    13
구로구     11
영등포구     8
노원구      8
은평구      6
강동구      5
송파구      2
Name: count, dtype: int64

In [59]:
len(data['행정구역'])

569

In [60]:
seoul_districts = [
    "강남구", "강동구", "강북구", "강서구", "관악구", "광진구", "구로구", "금천구", "노원구", "도봉구",
    "동대문구", "동작구", "마포구", "서대문구", "서초구", "성동구", "성북구", "송파구", "양천구",
    "영등포구", "용산구", "은평구", "종로구", "중구", "중랑구"
]


In [61]:
data = data[data['행정구역'].isin(seoul_districts)]

In [62]:
# 25개 구에 대한 빈 데이터프레임 생성
district_dfs = {district: pd.DataFrame(columns=data.columns) for district in seoul_districts}

for i in tqdm(range(len(data))):
    district = data['행정구역'].iloc[i]
    
    # district_dfs[district]가 비어 있는 경우 처리
    if district not in district_dfs or district_dfs[district].empty:
        district_dfs[district] = data.iloc[[i]].copy()  # 기존 DataFrame 복사
    else:
        district_dfs[district] = pd.concat([district_dfs[district], data.iloc[[i]]], ignore_index=True)

100%|██████████████████████████████████████| 569/569 [00:00<00:00, 10833.37it/s]


In [64]:
# 각 구별 데이터 저장 
for district, df in district_dfs.items():
    df['열선의_id'] = range(1, len(df) + 1) 
    df['찾은_도로의_id'] = ""
    
    df.insert(0, '찾은_도로의_id', df.pop('찾은_도로의_id'))
    df.insert(0, '열선의_id', df.pop('열선의_id'))
    df = df.reset_index(drop=True) 
    df.index = df.index + 1
    
    try :
        df_len = len(df)
        save_file_url = f"./make_file/gu_division/{district}_{df_len}_열선_데이터.csv"
        df.to_csv(save_file_url, encoding="utf-8", index=False)
        print(f"{save_file_url} 파일 저장 완료")
    except OSError as e :
        print(e)

./make_file/gu_division/강남구_59_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/강동구_5_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/강북구_14_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/강서구_13_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/관악구_26_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/광진구_33_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/구로구_11_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/금천구_15_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/노원구_8_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/도봉구_36_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/동대문구_13_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/동작구_45_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/마포구_21_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/서대문구_26_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/서초구_19_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/성동구_48_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/성북구_53_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/송파구_2_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/양천구_17_열선_데이터.csv 파일 저장 완료
./make_file/gu_division/영등포구_8_열